# Monitoring Kafka and Questdb

To monitor data, you usually have an agent running on your servers that collects metrics and then ingest into a time-series database. There are many agents to choose from, but a popular option that works well with Kafka and QuestDB is the Telegraf Agent. Telegraf supports many input plugins as metrics origin, many output plugins as metrics destination, and supports aggregators and transforms between input and output.

In this template, we have a telegraf configuration in the `./monitoring/telegraf` folder. That configuration reads metrics from the questdb monitoring endpoint (in prometheus format) `questdb:9003/metrics` and from the Kafka MX metrics server that we are exposing through a plugin on `http://broker:8778/jolokia`. After collecting the metrics and applying some filtering and transforms, metrics are then written into several tables in QuestDB.

On a production environment you would probably want to store metrics on a different server, but for this template we are storing the metrics in the same QuestDB instance where we store the user data.

We are not providing any monitoring dashboard on this template, but feel free to explore the metrics on this notebook, or even better at [http://localhost:9000](http://localhost:9000), and then try to create a Grafana dashboard at [http://localhost:3000](http://localhost:3000)

You can see the monitoring tables with this script

In [1]:
import psycopg2 as pg
import time
import json
from psycopg2.extras import RealDictCursor
import os

# Fetch environment variables with defaults
host = os.getenv('QDB_CLIENT_HOST', 'questdb')
port = os.getenv('QDB_CLIENT_PORT', '8812')
user = os.getenv('QDB_CLIENT_USER', 'admin')
password = os.getenv('QDB_CLIENT_PASSWORD', 'quest')

# Create the connection string using the environment variables or defaults
conn_str = f'user={user} password={password} host={host} port={port} dbname=qdb'

connection = pg.connect(conn_str)

# Open a cursor to perform database operations
cur = connection.cursor(cursor_factory=RealDictCursor)

#Query the database and obtain data as Python objects.
cur.execute('show tables;')
records = cur.fetchall()
for row in records:
     print(row)




RealDictRow([('table_name', 'kafka_cluster')])
RealDictRow([('table_name', 'trades')])
RealDictRow([('table_name', 'kafka_java_runtime')])
RealDictRow([('table_name', 'kafka_topic')])
RealDictRow([('table_name', 'prometheus')])
RealDictRow([('table_name', 'kafka_partition')])
RealDictRow([('table_name', 'github_events')])


As you can see, apart from any analytics tables, we have the `kafka_cluster`, `kafka_java_runtime`, `kafka_partition`, and `kafka_topic` tables to monitor Kafka, and the `prometheus` table to monitor QuestDB.

Let's explore these tables


In [21]:
#Query the database and obtain data as Python objects.
cur.execute('select * from kafka_java_runtime limit -1;')
records = cur.fetchall()
for row in records:
     print(row)

RealDictRow([('system', 'kafka_jvm'), ('Uptime', 4956010.0), ('G1 Old GenerationCollectionCount', 0.0), ('G1 Young GenerationCollectionTime', 2621.0), ('ThreadCount', 87.0), ('LoadedClassCount', 7582.0), ('TotalLoadedClassCount', 7582.0), ('G1 Old GenerationCollectionTime', 0.0), ('G1 Young GenerationCollectionCount', 114.0), ('TotalStartedThreadCount', 89.0), ('PeakThreadCount', 87.0), ('DaemonThreadCount', 43.0), ('UnloadedClassCount', 0.0), ('timestamp', datetime.datetime(2024, 2, 2, 19, 55, 30, 15000))])


Garbage Collector statistics in 1 minute intervals

In [27]:
#Query the database and obtain data as Python objects.
cur.execute('select timestamp, avg(`G1 Young GenerationCollectionTime`) as GC_1m from kafka_java_runtime sample by 1m align to calendar limit -5')
records = cur.fetchall()
for row in records:
     print(row)

RealDictRow([('timestamp', datetime.datetime(2024, 2, 2, 20, 0)), ('GC_1m', 2747.0)])
RealDictRow([('timestamp', datetime.datetime(2024, 2, 2, 20, 1)), ('GC_1m', 2830.5)])
RealDictRow([('timestamp', datetime.datetime(2024, 2, 2, 20, 2)), ('GC_1m', 2859.0)])
RealDictRow([('timestamp', datetime.datetime(2024, 2, 2, 20, 3)), ('GC_1m', 2902.0)])
RealDictRow([('timestamp', datetime.datetime(2024, 2, 2, 20, 4)), ('GC_1m', 2910.0)])


Some topic statistics per minute

In [28]:
#Query the database and obtain data as Python objects.
cur.execute("select timestamp, AVG(BytesOutPerSec_github_events_Count) as bytesOut, AVG(TotalFetchRequestsPerSec_github_events_Count) AS requests from kafka_topic where topic = 'github_events' SAMPLE BY 1m align to calendar limit -5;")
records = cur.fetchall()
for row in records:
     print(row)


RealDictRow([('timestamp', datetime.datetime(2024, 2, 2, 19, 52)), ('bytesOut', 1642013.0), ('requests', 19630.0)])
RealDictRow([('timestamp', datetime.datetime(2024, 2, 2, 19, 54)), ('bytesOut', 1693265.0), ('requests', 20189.0)])
RealDictRow([('timestamp', datetime.datetime(2024, 2, 2, 19, 57)), ('bytesOut', 1772884.0), ('requests', 21040.0)])
RealDictRow([('timestamp', datetime.datetime(2024, 2, 2, 20, 1)), ('bytesOut', 1866938.0), ('requests', 22040.0)])
RealDictRow([('timestamp', datetime.datetime(2024, 2, 2, 20, 3)), ('bytesOut', 1923234.0), ('requests', 22627.0)])


Some QuestDB metrics

In [30]:
#Query the database and obtain data as Python objects.
cur.execute("select SUM(questdb_memory_malloc_count) as malloc, SUM(questdb_jvm_minor_gc_count_total) as GC from prometheus  SAMPLE by 1m ALIGN TO CALENDAR limit -10;")
records = cur.fetchall()
for row in records:
     print(row)



RealDictRow([('malloc', 201772.0), ('GC', 14.0)])
RealDictRow([('malloc', 214739.0), ('GC', 14.0)])
RealDictRow([('malloc', 227711.0), ('GC', 14.0)])
RealDictRow([('malloc', 240713.0), ('GC', 14.0)])
RealDictRow([('malloc', 253800.0), ('GC', 14.0)])
RealDictRow([('malloc', 266965.0), ('GC', 14.0)])
RealDictRow([('malloc', 280190.0), ('GC', 14.0)])
RealDictRow([('malloc', 293385.0), ('GC', 14.0)])
RealDictRow([('malloc', 306646.0), ('GC', 14.0)])
RealDictRow([('malloc', 158324.0), ('GC', 7.0)])


In [6]:
cur.close()
connection.close()